### Дипломная работа Александра Соколова

#### Градиентный бустинг (CatBoost + GPU)
Кернел 4 из 5 в разделе ML (отредактирован 21.04.2021)
---

# 1. Импорт библиотек, инициализация глобальных констант
## 1.1. Импорт библиотек

In [ ]:
import pandas as pd
import numpy as np
import os
import tqdm
import pickle


from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import roc_auc_score

from catboost import CatBoostClassifier, Pool
os.environ["CUDA_VISIBLE_DEVICES"] = '0'

np.warnings.filterwarnings('ignore')

## 1.2. Глобальные константы

In [ ]:
# CURRENT_DIR = './'  # имя текущей директории для локальной машины 
CURRENT_DIR = '../'  # имя текущей директории для каггл

PATH_TO_WORKDIR = CURRENT_DIR + 'working/'

In [ ]:
!pip freeze > requirements.txt

## 1.3. Проверка подключения и параметров GPU

In [ ]:
!nvidia-smi

# 2. Импорт предобработанных данных
---
предобработка осуществлялась в [первом кернеле](https://www.kaggle.com/sokolovaleks/sf-dst-10-diplom-1-ml-sokolov)

In [ ]:
merged_train_data = pd.read_csv('../input/alfabattle2-sandbox/preproc_data_for_boosting/preproc_data_for_boosting/merged_data.csv')
merged_test_data = pd.read_csv('../input/alfabattle2-sandbox/preproc_data_for_boosting/preproc_data_for_boosting/merged_test_data.csv')

In [ ]:
features = [x for x in merged_train_data.columns if x not in ['app_id', 'flag']]

# 3. Разбиваем тренировочную выборку
---
Важно выделять валидационную выборку, чтобы контролировать обучение и не переобучаться. 

In [ ]:
targets = merged_train_data.flag.values

cv = KFold(n_splits=5, random_state=100, shuffle=True)

# 4. CatBoost + CV

In [ ]:
name_model = 'CatBoost'

In [ ]:
!mkdir model_CatBoost

In [ ]:
%%time

oof = np.zeros(len(merged_train_data))
train_preds = np.zeros(len(merged_train_data))

models = []

tree_params = {
    'max_depth': 5,
    'eval_metric': 'AUC',
    'loss_function': 'Logloss',
    'random_state': 100,
    'l2_leaf_reg': 1,
    'task_type': 'GPU' # если имеется GPU с CUDA, то можно ее задействовать, чтобы на порядок ускорить обучение
}


for fold_, (train_idx, val_idx) in enumerate(cv.split(merged_train_data, targets), 1):
    print(f'Началось обучение на фолде номер:= {fold_}.')
    model = CatBoostClassifier(**tree_params)    
    
    train, val = merged_train_data.iloc[train_idx], merged_train_data.iloc[val_idx]
    
    train_pool = Pool(train[features], train.flag.values)
    val_pool = Pool(val[features], val.flag.values)
    

    model.fit(train_pool, eval_set=[val_pool], early_stopping_rounds=100, verbose_eval=50, use_best_model=True, plot=False)
    
    oof[val_idx] = model.predict_proba(val_pool)[:, 1]
    
    train_preds[train_idx] += model.predict_proba(train_pool)[:, 1] / (cv.n_splits-1)
    models.append(model)
    
    file_name_model = f'model_CatBoost/model_{name_model}_{fold_}'
    model.save_model(PATH_TO_WORKDIR + file_name_model)
    print(f'Обучение на фолде номер:= {fold_} завершилось.')

In [ ]:
file_name_pickle = f'model_CatBoost/feats_model{name_model}.pickle'
with open(PATH_TO_WORKDIR + file_name_pickle, 'wb') as f:
    pickle.dump(features, f)

In [ ]:
'Train roc-auc model CatBoost', roc_auc_score(targets, train_preds)

In [ ]:
'CV roc-auc  model CatBoost', roc_auc_score(targets, oof)

In [ ]:
score = np.zeros(len(merged_test_data))

test_pool = Pool(merged_test_data[features])

for model in tqdm.tqdm_notebook(models):
    score += model.predict_proba(test_pool)[:, 1] / len(models)
    
submission = pd.DataFrame({
    'app_id' : merged_test_data.app_id.values,
    'score': score
}) 
submission.to_csv(PATH_TO_WORKDIR + f'sub_model{name_model}.csv', index=None) # ~ 0.732 на public test